In [2]:
import os

def generate_medical_report(mri_file, output_file="medical_report2.txt"):
    # Run fslstats command and capture the output
    command = f"fslstats {mri_file} -R -M -V -P 50 -S"
    result = os.popen(command).read().strip().split()

    if len(result) < 6:
        print("Error: Unable to extract statistics.")
        return

    # Extract values
    min_intensity = result[0]
    max_intensity = result[1]
    mean_intensity = result[2]
    brain_volume = result[3]
    median_intensity = result[4]
    std_dev = result[5]

    # Generate a structured report
    report = f"""
    🏥 **MRI Scan Medical Report**
    ------------------------------------
    📌 **Scan File:** {mri_file}

    📊 **Statistical Analysis:**
    - 🔹 Minimum Intensity: {min_intensity}
    - 🔹 Maximum Intensity: {max_intensity}
    - 🔹 Mean Intensity: {mean_intensity}
    - 🔹 Brain Volume: {brain_volume} mm³
    - 🔹 Median Intensity: {median_intensity}
    - 🔹 Standard Deviation: {std_dev}

    📋 **Medical Interpretation:**
    - ✅ Normal brain tissue usually has intensity values within a specific range.
    - ⚠️ Higher intensity regions may indicate lesions or abnormal growth.
    - ⚠️ Lower mean intensity could indicate atrophy or degeneration.
    
    📌 **Recommendation:**
    - If abnormalities are suspected, further examination via radiology or a neurologist consultation is advised.
    """
    
    # Save the report to a file
    with open(output_file, "w") as f:
        f.write(report)

    print(f"✅ Medical report saved to {output_file}")

# Example usage
mri_scan_file = "/Users/malhar.inamdar/Desktop/report/images/OAS1_0043_MR1_mpr-2_anon.img"
generate_medical_report(mri_scan_file)

✅ Medical report saved to medical_report2.txt


In [1]:
malhar.inamdar@Malhars-MacBook-Air report % fslstats fast_output_pve_1.nii.gz -V

# 536828
# 671035.000000
# The number of nonzero voxels
# The total volume in mm³ (if the header has voxel dimensions).

#grey matter volume

#white matter volume:
# 419446 524307.500000 


#cerebrospinal fluid volume:

245724 307155.000000 

SyntaxError: invalid syntax (705887267.py, line 1)

In [3]:
import os

def generate_medical_report(mri_file, output_file="medical_report3.txt"):
    # ---------------------------------------
    # 1. Basic Statistical Analysis with fslstats
    # ---------------------------------------
    # This command computes:
    #   - -R: minimum and maximum intensities
    #   - -M: mean intensity
    #   - -V: voxel count (and volume in mm³ if header info is available)
    #   - -P 50: 50th percentile (median)
    #   - -S: standard deviation
    #
    # (Note: fslstats -V normally returns two numbers: voxel_count and volume.
    #  Here we assume that result[3] represents the volume in mm³ as per your original code.)
    command = f"fslstats {mri_file} -R -M -V -P 50 -S"
    result = os.popen(command).read().strip().split()

    if len(result) < 6:
        print("Error: Unable to extract basic statistics from the MRI scan.")
        return

    # Extract basic statistics (values are returned as strings)
    min_intensity    = result[0]
    max_intensity    = result[1]
    mean_intensity   = result[2]
    brain_volume     = result[3]  # volume in mm³ (as per your original code usage)
    median_intensity = result[4]
    std_dev          = result[5]

    # ---------------------------------------
    # 2. Tissue Segmentation using FSL FAST
    # ---------------------------------------
    # FAST requires both the image and its header (hdr) file to compute voxel dimensions.
    # The FAST command below will segment the T1-weighted image into 3 tissue types.
    #
    # We set an output prefix based on the MRI file name.
    base, ext = os.path.splitext(mri_file)
    fast_prefix = base + "_fast"
    
    fast_command = f"fast -t 1 -o {fast_prefix} {mri_file}"
    print("Running FAST segmentation. This may take a few minutes...")
    os.system(fast_command)
    
    # FAST creates a segmentation file (with hard labels) named: <fast_prefix>_seg.nii.gz
    seg_file = fast_prefix + "_seg.nii.gz"
    
    if not os.path.exists(seg_file):
        print("Error: FAST segmentation failed or the segmentation file was not found.")
        return

    # Using the segmentation file, extract tissue-specific volumes.
    # The segmentation file contains integer labels:
    #   CSF = 1, GM = 2, WM = 3.
    # We use fslstats with a lower (-l) and upper (-u) threshold to select voxels
    # belonging to each label. The "-V" option returns two numbers: the voxel count
    # and the computed volume (in mm³) for that tissue.
    csf_stats = os.popen(f"fslstats {seg_file} -l 0.5 -u 1.5 -V").read().strip().split()
    gm_stats  = os.popen(f"fslstats {seg_file} -l 1.5 -u 2.5 -V").read().strip().split()
    wm_stats  = os.popen(f"fslstats {seg_file} -l 2.5 -u 3.5 -V").read().strip().split()

    if len(csf_stats) < 2 or len(gm_stats) < 2 or len(wm_stats) < 2:
        print("Error: Unable to extract tissue volumes from the segmentation file.")
        return

    # We use the second value returned by -V (i.e. the computed volume in mm³)
    csf_volume = csf_stats[1]
    gm_volume  = gm_stats[1]
    wm_volume  = wm_stats[1]

    # ---------------------------------------
    # 3. Generate the Medical Report
    # ---------------------------------------
    report = f"""
    🏥 **MRI Scan Medical Report**
    ------------------------------------
    📌 **Scan File:** {mri_file}

    📊 **Basic Statistical Analysis:**
    - 🔹 Minimum Intensity: {min_intensity}
    - 🔹 Maximum Intensity: {max_intensity}
    - 🔹 Mean Intensity: {mean_intensity}
    - 🔹 Brain Volume: {brain_volume} mm³
    - 🔹 Median Intensity: {median_intensity}
    - 🔹 Standard Deviation: {std_dev}

    🧠 **Tissue Volume Analysis (via FAST segmentation):**
    - 💧 Cerebrospinal Fluid (CSF) Volume: {csf_volume} mm³
    - 🧠 Grey Matter (GM) Volume: {gm_volume} mm³
    - ⚪ White Matter (WM) Volume: {wm_volume} mm³

    📋 **Medical Interpretation:**
    - ✅ Normal brain tissue usually exhibits intensity values within a specific range.
    - ⚠️ Regions with unusually high intensities may indicate lesions or abnormal growth.
    - ⚠️ Lower mean intensities could be associated with atrophy or degeneration.
    
    📌 **Recommendation:**
    - If any abnormalities are suspected, further evaluation (e.g., with additional radiological studies or a neurologist consultation) is advised.
    """
    
    # Save the report to a file
    with open(output_file, "w") as f:
        f.write(report)

    print(f"✅ Medical report saved to {output_file}")

# ---------------------------------------
# Example usage:
# Make sure that the MRI image (.img) and its header (.hdr) are in the same directory.
# ---------------------------------------
mri_scan_file = "/Users/malhar.inamdar/Desktop/report/images/OAS1_0043_MR1_mpr-3_anon.img"
generate_medical_report(mri_scan_file)


Running FAST segmentation. This may take a few minutes...
✅ Medical report saved to medical_report3.txt


In [3]:
import os
from fpdf import FPDF

def generate_medical_report(mri_file, output_file="medical_report3.pdf"):
    # ---------------------------------------
    # 1. Basic Statistical Analysis with fslstats
    # ---------------------------------------
    # This command computes:
    #   - -R: minimum and maximum intensities
    #   - -M: mean intensity
    #   - -V: voxel count (and volume in mm³ if header info is available)
    #   - -P 50: 50th percentile (median)
    #   - -S: standard deviation
    #
    # (Note: fslstats -V normally returns two numbers: voxel_count and volume.
    #  Here we assume that result[3] represents the volume in mm³ as per your original code.)
    command = f"fslstats {mri_file} -R -M -V -P 50 -S"
    result = os.popen(command).read().strip().split()

    if len(result) < 6:
        print("Error: Unable to extract basic statistics from the MRI scan.")
        return

    # Extract basic statistics (values are returned as strings)
    min_intensity    = result[0]
    max_intensity    = result[1]
    mean_intensity   = result[2]
    brain_volume     = result[3]  # volume in mm³ (as per your original code usage)
    median_intensity = result[4]
    std_dev          = result[5]

    # ---------------------------------------
    # 2. Tissue Segmentation using FSL FAST
    # ---------------------------------------
    # FAST requires both the image and its header (hdr) file to compute voxel dimensions.
    # The FAST command below will segment the T1-weighted image into 3 tissue types.
    #
    # We set an output prefix based on the MRI file name.
    base, ext = os.path.splitext(mri_file)
    fast_prefix = base + "_fast"
    
    fast_command = f"fast -t 1 -o {fast_prefix} {mri_file}"
    print("Running FAST segmentation. This may take a few minutes...")
    os.system(fast_command)
    
    # FAST creates a segmentation file (with hard labels) named: <fast_prefix>_seg.nii.gz
    seg_file = fast_prefix + "_seg.nii.gz"
    
    if not os.path.exists(seg_file):
        print("Error: FAST segmentation failed or the segmentation file was not found.")
        return

    # Using the segmentation file, extract tissue-specific volumes.
    # The segmentation file contains integer labels:
    #   CSF = 1, GM = 2, WM = 3.
    # We use fslstats with a lower (-l) and upper (-u) threshold to select voxels
    # belonging to each label. The "-V" option returns two numbers: the voxel count
    # and the computed volume (in mm³) for that tissue.
    csf_stats = os.popen(f"fslstats {seg_file} -l 0.5 -u 1.5 -V").read().strip().split()
    gm_stats  = os.popen(f"fslstats {seg_file} -l 1.5 -u 2.5 -V").read().strip().split()
    wm_stats  = os.popen(f"fslstats {seg_file} -l 2.5 -u 3.5 -V").read().strip().split()

    if len(csf_stats) < 2 or len(gm_stats) < 2 or len(wm_stats) < 2:
        print("Error: Unable to extract tissue volumes from the segmentation file.")
        return

    # We use the second value returned by -V (i.e. the computed volume in mm³)
    csf_volume = csf_stats[1]
    gm_volume  = gm_stats[1]
    wm_volume  = wm_stats[1]

    # ---------------------------------------
    # 3. Generate the Medical Report
    # ---------------------------------------
    report = f"""
    🏥 **MRI Scan Medical Report**
    ------------------------------------
    📌 **Scan File:** {mri_file}

    📊 **Basic Statistical Analysis:**
    - 🔹 Minimum Intensity: {min_intensity}
    - 🔹 Maximum Intensity: {max_intensity}
    - 🔹 Mean Intensity: {mean_intensity}
    - 🔹 Brain Volume: {brain_volume} mm³
    - 🔹 Median Intensity: {median_intensity}
    - 🔹 Standard Deviation: {std_dev}

    🧠 **Tissue Volume Analysis (via FAST segmentation):**
    - 💧 Cerebrospinal Fluid (CSF) Volume: {csf_volume} mm³
    - 🧠 Grey Matter (GM) Volume: {gm_volume} mm³
    - ⚪ White Matter (WM) Volume: {wm_volume} mm³

    📋 **Medical Interpretation:**
    - ✅ Normal brain tissue usually exhibits intensity values within a specific range.
    - ⚠️ Regions with unusually high intensities may indicate lesions or abnormal growth.
    - ⚠️ Lower mean intensities could be associated with atrophy or degeneration.
    
    📌 **Recommendation:**
    - If any abnormalities are suspected, further evaluation (e.g., with additional radiological studies or a neurologist consultation) is advised.
    """
    
    # Save the report to a file
    with open(output_file, "w") as f:
        f.write(report)

    print(f"✅ Medical report saved to {output_file}")

# ---------------------------------------
# Example usage:
# Make sure that the MRI image (.img) and its header (.hdr) are in the same directory.
# ---------------------------------------
mri_scan_file = "/Users/malhar.inamdar/Desktop/report/images/OAS1_0043_MR1_mpr-4_anon.img"
generate_medical_report(mri_scan_file)


Running FAST segmentation. This may take a few minutes...
✅ Medical report saved to medical_report3.pdf


In [4]:
import nibabel as nib
import numpy as np
import cv2
import os

# Load the .img file
img_path = "/Users/malhar.inamdar/Desktop/report/images/OAS1_0043_MR1_mpr-1_anon.img"
output_folder = "/Users/malhar.inamdar/Desktop/report/converted_jpgs"

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Load MRI scan
mri_img = nib.load(img_path)
mri_data = mri_img.get_fdata()  # Get pixel values as a NumPy array

# Normalize the MRI scan for visualization
def normalize_image(image):
    image = (image - np.min(image)) / (np.max(image) - np.min(image)) * 255
    return image.astype(np.uint8)

# Select the middle slice
slice_index = mri_data.shape[2] // 2  
slice_2d = normalize_image(mri_data[:, :, slice_index])

# Rotate counterclockwise (left) before saving
slice_2d = cv2.rotate(slice_2d, cv2.ROTATE_90_COUNTERCLOCKWISE)

# Save the image as JPEG
output_path = os.path.join(output_folder, "mri_scan11.jpg")
cv2.imwrite(output_path, slice_2d)

print(f"Converted MRI slice saved at: {output_path}")


Converted MRI slice saved at: /Users/malhar.inamdar/Desktop/report/converted_jpgs/mri_scan11.jpg
